In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-addon-cutting

*Tantiya sa paggamit: isang minuto sa Eagle processor (PAALALA: Ito ay tantiya lamang. Maaaring mag-iba ang inyong runtime.)*

## Background

Ang circuit-knitting ay isang pangkalahatang termino na sumasaklaw sa iba't ibang pamamaraan ng paghahati ng circuit sa maraming mas maliit na subcircuits na may mas kaunting gates at/o qubits. Ang bawat isa sa mga subcircuits ay maaaring isagawa nang independiyente at ang panghuling resulta ay nakukuha sa pamamagitan ng ilang classical post-processing sa kinalabasan ng bawat subcircuit. Ang pamamaraang ito ay maaaring ma-access sa [circuit cutting Qiskit addon](https://qiskit.github.io/qiskit-addon-cutting/index.html), ang detalyadong paliwanag ng pamamaraan ay ibinigay sa [docs](https://qiskit.github.io/qiskit-addon-cutting/explanation/index.html) kasama ng iba pang [introductory material](https://qiskit.github.io/qiskit-addon-cutting/tutorials/index.html).

Ang notebook na ito ay tumatalakay sa isang pamamaraang tinatawag na <b>wire cutting</b> kung saan ang circuit ay pinaghahati sa kahabaan ng wire [\[1\], \[2\]](#references). Tandaan na ang paghahati ay simple sa mga classical circuits dahil ang kinalabasan sa punto ng partition ay maaaring matukoy nang deterministiko, at ito ay 0 o 1 lamang. Gayunpaman, ang estado ng qubit sa punto ng pagputol ay, sa pangkalahatan, isang mixed state. Samakatuwid, ang bawat subcircuit ay kailangang sukatin nang maraming beses sa iba't ibang basis (karaniwang isang tomographically complete set ng basis tulad ng Pauli basis [\[3\], \[4\]](#references) at naaangkop na inihanda sa kanyang eigenstate. Ang Figure sa ibaba (<i>pasasalamat: PhD Thesis, Ritajit Majumdar</i>) ay nagpapakita ng halimbawa ng wire cutting para sa 4-qubit GHZ state sa tatlong subcircuits. Dito ang $M_j$ ay tumutukoy sa isang set ng basis (karaniwang Pauli X, Y at Z) at ang $P_i$ ay tumutukoy sa isang set ng eigenstates (karaniwang $|0\rangle$, $|1\rangle$, $|+\rangle$ at $|+i\rangle$).

![wc-1.png](../docs/images/tutorials/wire-cutting-to-improve-performance/0ce8857b-7f5f-400e-8536-6a496c724d50.avif)
![wc-2.png](../docs/images/tutorials/wire-cutting-to-improve-performance/cbce4455-4794-4c81-8630-3e3993e1b29f.avif)

Dahil ang bawat subcircuit ay may mas kaunting qubits at/o gates, inaasahang mas hindi sila madaling maapektuhan ng noise. Ang notebook na ito ay nagpapakita ng halimbawa kung saan ang pamamaraang ito ay maaaring gamitin upang epektibong pigilin ang noise sa sistema.

## Mga Kinakailangan

Bago simulan ang tutorial na ito, tiyaking mayroon kayong mga sumusunod na naka-install:

- Qiskit SDK v2.0 o mas bago, na may [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization) support
- Qiskit Runtime v0.22 o mas bago ( `pip install qiskit-ibm-runtime` )
- Circuit cutting Qiskit addon v0.9.0 o mas bago (`pip install qiskit-addon-cutting`)

Isasaalang-alang natin ang isang Many Body Localization (MBL) circuit para sa notebook na ito. Ang MBL circuit ay isang hardware-efficient circuit at may parameter na dalawang parameters na $\theta$ at $\vec{\phi}$. Kapag ang $\theta$ ay nakatakda sa $0$ at ang initial state ay inihanda sa $|0\rangle$ para sa lahat ng qubits, ang ideal expectation value ng $\langle Z_i \rangle$ ay $+1$ para sa bawat qubit site $i$ anuman ang mga halaga ng $\vec{\phi}$. Maaari ninyong tingnan ang mas maraming detalye tungkol sa MBL circuits sa <a href="https://arxiv.org/abs/2307.07552">papel na ito</a>.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


from qiskit.circuit import Parameter, ParameterVector, QuantumCircuit
from qiskit.quantum_info import PauliList, SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.result import sampled_expectation_value

from qiskit_addon_cutting.instructions import CutWire
from qiskit_addon_cutting import (
    cut_wires,
    expand_observables,
    partition_problem,
    generate_cutting_experiments,
    reconstruct_expectation_values,
)

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2, Batch


class MBLChainCircuit(QuantumCircuit):
    def __init__(
        self, num_qubits: int, depth: int, use_cut: bool = False
    ) -> None:
        super().__init__(
            num_qubits, name=f"MBLChainCircuit<{num_qubits}, {depth}>"
        )
        evolution = MBLChainEvolution(num_qubits, depth, use_cut)
        self.compose(evolution, inplace=True)


class MBLChainEvolution(QuantumCircuit):
    def __init__(self, num_qubits: int, depth: int, use_cut) -> None:
        super().__init__(
            num_qubits, name=f"MBLChainEvolution<{num_qubits}, {depth}>"
        )

        theta = Parameter("θ")
        phis = ParameterVector("φ", num_qubits)

        for layer in range(depth):
            layer_parity = layer % 2
            # print("layer parity", layer_parity)
            for qubit in range(layer_parity, num_qubits - 1, 2):
                # print(qubit)
                self.cz(qubit, qubit + 1)
                self.u(theta, 0, np.pi, qubit)
                self.u(theta, 0, np.pi, qubit + 1)
                if (
                    use_cut
                    and layer_parity == 0
                    and (
                        qubit == num_qubits // 2 - 1
                        or qubit == num_qubits // 2
                    )
                ):
                    self.append(CutWire(), [num_qubits // 2])
                if use_cut and layer < depth - 1 and layer_parity == 1:
                    if qubit == num_qubits // 2:
                        self.append(CutWire(), [qubit])
            for qubit in range(num_qubits):
                self.p(phis[qubit], qubit)

## Bahagi I. Halimbawa ng maliit na sukat
### Hakbang 1: I-map ang classical inputs sa quantum problem
Una, bubuo tayo ng template circuit na walang partikular na parameter values. Nagbibigay din tayo ng mga placeholders, na tinatawag na `CutWire`, upang markahan ang posisyon ng mga pagputol. Para sa halimbawa ng maliit na sukat, isasaalang-alang natin ang 10-qubit MBL circuit.

In [68]:
num_qubits = 10
depth = 2
mbl = MBLChainCircuit(num_qubits, depth)
mbl.draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/wire-cutting/extracted-outputs/9c7939a1-6b70-4dad-b873-5f34d67551c4-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/wire-cutting/extracted-outputs/9c7939a1-6b70-4dad-b873-5f34d67551c4-0.avif)

Alalahanin na ang layunin natin ay mahanap ang expectation value ng observable na $\frac{1}{n}\sum_{i=1} ^n Z_i$ kapag $\theta=0$. Maglalagay tayo ng ilang random na halaga para sa parameter na $\vec{\phi}$.

In [69]:
phis = list(np.random.rand(mbl.num_parameters - 1))
theta = [0]
params = theta + phis
params

[0,
 0.2376615174332788,
 0.28244289857682414,
 0.019248960591717768,
 0.46140600996102477,
 0.31408025180068433,
 0.718184005135733,
 0.991153920182475,
 0.09289485768301442,
 0.8857848280067783,
 0.6177529765767047]

Now we annotate the circuit for cutting by inserting proper **CutWire** to create two roughly equal cuts. We set `use_cut=True` in the function, and allow it to annotate after $\frac{n}{2}$ qubits, $n$ being the number of qubits in the original circuit.

In [70]:
mbl_cut = MBLChainCircuit(num_qubits, depth, use_cut=True)
mbl_cut.assign_parameters(params, inplace=True)
mbl_cut.draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/wire-cutting/extracted-outputs/31844134-514b-46ea-85f9-133e432f053f-0.avif" alt="Output of the previous code cell" />

Ngayon ay markahan natin ang circuit para sa pagputol sa pamamagitan ng paglalagay ng wastong **CutWire** upang lumikha ng dalawang halos pantay na pagputol. Itinakda natin ang `use_cut=True` sa function, at papayagan itong markahan pagkatapos ng $\frac{n}{2}$ qubits, kung saan ang $n$ ay ang bilang ng mga qubits sa orihinal na circuit.

In [ ]:
mbl_move = cut_wires(mbl_cut)

![Output of the previous code cell](../docs/images/tutorials/wire-cutting/extracted-outputs/31844134-514b-46ea-85f9-133e432f053f-0.avif)

### Hakbang 2: I-optimize ang problema para sa quantum hardware execution
Susunod, puputulin natin ang circuit sa dalawang mas maliit na subcircuits. Para sa halimbawang ito, manatili tayo sa 2 subcircuits lamang. Para dito, gagamitin natin ang <a href="https://qiskit.github.io/qiskit-addon-cutting/">Qiskit Addon: Circuit Cutting</a>.
#### Putulin ang circuit sa mas maliit na subcircuits
Ang pagputol ng wire sa isang punto ay nagdaragdag ng isa sa bilang ng qubit. Bukod sa orihinal na qubit, mayroon na ngayong karagdagang qubit bilang placeholder sa circuit pagkatapos ng pagputol. Ang sumusunod na larawan ay nagbibigay ng representasyon:

![wc-4.png](../docs/images/tutorials/wire-cutting-to-improve-performance/dfc5f923-e507-4873-888e-d90e1618be3a.avif)

Ang Addon na ito ay gumagamit ng function na `cut_wires` upang isaalang-alang ang mga karagdagang qubits na lumilitaw dahil sa pagputol.

In [ ]:
observable = PauliList(
    ["I" * i + "Z" + "I" * (num_qubits - i - 1) for i in range(num_qubits)]
)
observable

PauliList(['ZIIIIIIIII', 'IZIIIIIIII', 'IIZIIIIIII', 'IIIZIIIIII',
           'IIIIZIIIII', 'IIIIIZIIII', 'IIIIIIZIII', 'IIIIIIIZII',
           'IIIIIIIIZI', 'IIIIIIIIIZ'])

#### Lumikha at palawakin ang mga observables
Ngayon ay bubuo tayo ng observable na $M_z = \frac{1}{n}\sum_{i=1}^n \langle Z_i \rangle$. Dahil ang ideal na kinalabasan ng $\langle Z_i \rangle$ para sa bawat $i$ ay $+1$, ang ideal na kinalabasan ng $M_z$ ay $+1$ din.

In [ ]:
new_obs = expand_observables(observable, mbl, mbl_move)
new_obs

PauliList(['ZIIIIIIIIII', 'IZIIIIIIIII', 'IIZIIIIIIII', 'IIIZIIIIIII',
           'IIIIZIIIIII', 'IIIIIIZIIII', 'IIIIIIIZIII', 'IIIIIIIIZII',
           'IIIIIIIIIZI', 'IIIIIIIIIIZ'])

Note that each observable has now expanded to accommodate seven qubits, as in the circuit with `Move` operation, instead of the original 6 qubits. Next, partition the circuit into two subcircuits.

In [ ]:
partitioned_problem = partition_problem(circuit=mbl_move, observables=new_obs)

Gayunpaman, pansinin na ang bilang ng mga qubits sa circuit ay tumaas pagkatapos ng paglalagay ng virtual 2-qubit `Move` operations pagkatapos ng pagputol. Samakatuwid, kailangan din nating palawakin ang mga observables sa pamamagitan ng paglalagay ng mga identities upang mag-assert sa kasalukuyang circuit.

In [9]:
subcircuits = partitioned_problem.subcircuits

In [10]:
subcircuits[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/wire-cutting/extracted-outputs/c10af39c-88fe-4605-975c-5bf0e21ee4c4-0.avif" alt="Output of the previous code cell" />

In [11]:
subcircuits[1].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/wire-cutting/extracted-outputs/35920640-76e8-4af6-a252-ee6a22e9c26a-0.avif" alt="Output of the previous code cell" />

Tingnan natin ang mga subcircuits

In [12]:
subobservables = partitioned_problem.subobservables
subobservables

{0: PauliList(['IIIIII', 'IIIIII', 'IIIIII', 'IIIIII', 'IIIIII', 'IZIIII',
            'IIZIII', 'IIIZII', 'IIIIZI', 'IIIIIZ']),
 1: PauliList(['ZIIII', 'IZIII', 'IIZII', 'IIIZI', 'IIIIZ', 'IIIII', 'IIIII',
            'IIIII', 'IIIII', 'IIIII'])}

Note that each subcircuit leads to a number of samples. The reconstruction takes into account the outcome of each of these samples. Each of these samples is termed a `subexperiment`.

Extending the observable using the `Move` operation requires a `PauliList` data structure. We can also create the $M_z$ observable in the more generic `SparsePauliOp` data structure which will be useful later during reconstruction of the subexperiments.

In [ ]:
M_z = SparsePauliOp(
    ["I" * i + "Z" + "I" * (num_qubits - i - 1) for i in range(num_qubits)],
    coeffs=[1 / num_qubits] * num_qubits,
)
M_z

SparsePauliOp(['ZIIIIIIIII', 'IZIIIIIIII', 'IIZIIIIIII', 'IIIZIIIIII', 'IIIIZIIIII', 'IIIIIZIIII', 'IIIIIIZIII', 'IIIIIIIZII', 'IIIIIIIIZI', 'IIIIIIIIIZ'],
              coeffs=[0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j,
 0.1+0.j, 0.1+0.j])

In [ ]:
subexperiments, coefficients = generate_cutting_experiments(
    circuits=subcircuits,
    observables=subobservables,
    num_samples=np.inf,
)

![Output of the previous code cell](../docs/images/tutorials/wire-cutting/extracted-outputs/35920640-76e8-4af6-a252-ee6a22e9c26a-0.avif)

Ang mga observables ay pinaghati rin upang umangkop sa mga subcircuits

In [15]:
subexperiments[0][6].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/wire-cutting/extracted-outputs/749e8f27-9c83-48d8-bcf6-635c967bf10b-0.avif" alt="Output of the previous code cell" />

In [16]:
subexperiments[0][2].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/wire-cutting/extracted-outputs/987547e4-296a-41e4-ad82-41f4139a87a0-0.avif" alt="Output of the previous code cell" />

Pansinin na ang bawat subcircuit ay humahantong sa ilang samples. Isinasaalang-alang ng reconstruction ang kinalabasan ng bawat isa sa mga samples na ito. Ang bawat isa sa mga samples na ito ay tinatawag na `subexperiment`.
Ang pagpapalawak ng observable gamit ang `Move` operation ay nangangailangan ng `PauliList` data structure. Maaari din tayong lumikha ng $M_z$ observable sa mas generic na `SparsePauliOp` data structure na magiging kapaki-pakinabang mamaya sa panahon ng reconstruction ng mga subexperiments.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

Now we need to transpile each of the circuits in the subexperiments. For that we first create a pass manager, and then use it to transpile each of the circuits.

In [ ]:
pm = generate_preset_pass_manager(optimization_level=2, backend=backend)

In [19]:
isa_subexperiments = {
    label: pm.run(partition_subexpts)
    for label, partition_subexpts in subexperiments.items()
}

In [20]:
isa_subexperiments[0][0].draw("mpl", fold=-1, idle_wires=False)

<Image src="../docs/images/tutorials/wire-cutting/extracted-outputs/962c4745-a235-4ef8-b56b-d3026be67fb6-0.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives

Now we shall execute each circuit in subexperiment. `Qiskit-addon-cutting` uses `SamplerV2` to execute the subexperiments.

In [ ]:
with Batch(backend=backend) as batch:
    sampler = SamplerV2(mode=batch)
    jobs = {
        label: sampler.run(subsystem_subexpts, shots=2**12)
        for label, subsystem_subexpts in isa_subexperiments.items()
    }

![Output of the previous code cell](../docs/images/tutorials/wire-cutting/extracted-outputs/987547e4-296a-41e4-ad82-41f4139a87a0-0.avif)

#### I-transpile ang bawat subexperiment

Sa kasalukuyan ay kailangan nating i-transpile ang ating mga circuits bago isumite ang mga ito para sa execution. Samakatuwid, ita-transpile natin muna ang bawat circuit sa mga subexperiments.

In [22]:
# Retrieve results
results = {label: job.result() for label, job in jobs.items()}

In [ ]:
reconstructed_expval_terms = reconstruct_expectation_values(
    results,
    coefficients,
    subobservables,
)
reconstructed_expval = np.dot(reconstructed_expval_terms, M_z.coeffs).real
reconstructed_expval

0.9674376845359803

#### Cross verify

Let us now execute the circuit without cutting and check the outcome there. Note that for execution of the uncut circuit we can directly use `EstimatorV2` for calculating the expectation values. But we shall use the same `Primitive` throughout. So we shall use `SamplerV2` to get the probability distribution and calculate the expectation value using the `sampled_expectation_value` function.

First we need to transpile the uncut `mbl` circuit.

In [24]:
sampler = SamplerV2(mode=backend)

if mbl.num_clbits == 0:
    mbl.measure_all()
isa_mbl = pm.run(mbl)

Next we construct the `pub` and run the uncut circuit.

In [ ]:
pub = (isa_mbl, params)
uncut_job = sampler.run([pub])

In [25]:
uncut_counts = uncut_job.result()[0].data.meas.get_counts()

In [ ]:
uncut_expval = sampled_expectation_value(uncut_counts, M_z)
uncut_expval

0.9498046875000001

We note that the expectation value obtained via wire cutting is closer to the ideal value of $+1$ than the uncut one. Let us now scale up the size of the problem.

## Part II. Scale it up!

Previously, we showed the results for a 10-qubit MBL circuit. Next, we show that the improvement in expectation value is also obtained for larger circuits. To show that, we repeat the process for a 60-qubit MBL circuit.

### Step 1: Map classical inputs to a quantum problem

In [29]:
num_qubits = 60
depth = 2
mbl = MBLChainCircuit(num_qubits, depth)

We create a random set of values for $\vec{\phi}$

In [ ]:
phis = list(np.random.rand(mbl.num_parameters - 1))
theta = [0]
params = theta + phis

#### Cross verify
Isagawa natin ngayon ang circuit nang walang pagputol at suriin ang kinalabasan doon. Pansinin na para sa execution ng uncut circuit ay maaari tayong direktang gumamit ng `EstimatorV2` para sa pagkalkula ng expectation values. Ngunit gagamitin natin ang parehong `Primitive` sa buong proseso. Kaya gagamitin natin ang `SamplerV2` upang makuha ang probability distribution at kalkulahin ang expectation value gamit ang `sampled_expectation_value` function.

Una ay kailangan nating i-transpile ang uncut na `mbl` circuit.

In [ ]:
mbl_cut = MBLChainCircuit(num_qubits, depth, use_cut=True)
mbl_cut.assign_parameters(params, inplace=True)
mbl_cut.draw("mpl", fold=-1)

Susunod ay bubuo tayo ng `pub` at patakbuhin ang uncut circuit.

In [32]:
mbl_move = cut_wires(mbl_cut)

# Define observable
observable = PauliList(
    ["I" * i + "Z" + "I" * (num_qubits - i - 1) for i in range(num_qubits)]
)
new_obs = expand_observables(observable, mbl, mbl_move)

# Partition the circuit into subcircuits
partitioned_problem = partition_problem(circuit=mbl_move, observables=new_obs)

# Get subcircuits
subcircuits = partitioned_problem.subcircuits

In [ ]:
subobservables = partitioned_problem.subobservables

We also create a `SparsePauliOp` object for the observable with proper co-efficients.

In [ ]:
M_z = SparsePauliOp(
    ["I" * i + "Z" + "I" * (num_qubits - i - 1) for i in range(num_qubits)],
    coeffs=[1 / num_qubits] * num_qubits,
)

Next we generate the subexperiments and transpile each circuit in the subexperiment.

In [38]:
subexperiments, coefficients = generate_cutting_experiments(
    circuits=subcircuits,
    observables=subobservables,
    num_samples=np.inf,
)

In [39]:
isa_subexperiments = {
    label: pm.run(partition_subexpts)
    for label, partition_subexpts in subexperiments.items()
}

Lumikha tayo ng random na set ng mga halaga para sa $\vec{\phi}$

In [43]:
with Batch(backend=backend) as batch:
    sampler = SamplerV2(mode=batch)
    jobs = {
        label: sampler.run(subsystem_subexpts, shots=2**12)
        for label, subsystem_subexpts in isa_subexperiments.items()
    }

Susunod ay bubuo tayo ng cut circuit

In [44]:
# Retrieve results
results = {label: job.result() for label, job in jobs.items()}

In [45]:
reconstructed_expval_terms = reconstruct_expectation_values(
    results,
    coefficients,
    subobservables,
)
reconstructed_expval = np.dot(reconstructed_expval_terms, M_z.coeffs).real
reconstructed_expval

0.9631355921427409

#### Cross verify

As in the small scale example, we shall once more obtain the expectation value by executing the uncut circuit, and compare the result with circuit cutting. We shall use the `SamplerV2` to maintain uniformity in the use of Primitives.

In [48]:
sampler = SamplerV2(mode=backend)

if mbl.num_clbits == 0:
    mbl.measure_all()
isa_mbl = pm.run(mbl)

pub = (isa_mbl, params)
uncut_job = sampler.run([pub])

In [49]:
uncut_counts = uncut_job.result()[0].data.meas.get_counts()
uncut_expval = sampled_expectation_value(uncut_counts, M_z)
uncut_expval

0.9426757812499998

Susunod ay bubuo tayo ng mga subexperiments at ita-transpile ang bawat circuit sa subexperiment.

In [ ]:
ax = plt.gca()
methods = ["cut", "uncut"]
values = [reconstructed_expval, uncut_expval]

plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(y=1, color="k", linestyle="--")
ax.set_ylim([0.85, 1.02])
plt.text(0.3, 0.99, "Exact result")
plt.show()

<Image src="../docs/images/tutorials/wire-cutting/extracted-outputs/19dad6bb-544e-45b9-90c4-3555681e4e5b-0.avif" alt="Output of the previous code cell" />

#### Inference

We observe that both in the small and large scale problems wire cutting leads to a better result than the uncut one. Note that no error mitigation techniques have been used for these experiments. Therefore, the improvement in result that has been obtained is only due to wire cutting. It may be possible to further improve the results using different mitigation methods together with circuit cutting.

Moreover, in this notebook, we computed both the subcircuits on the same hardware. In [\[5\], \[6\]](#references), the authors shows a method to distribute the subcircuits on different hardware using noise information in order to maximize the noise suppression, and parallelize the process.

## Appendix: resource scaling consideration

The number of circuits to be executed increases with the number of cuts. Therefore, while many cuts can produce small subcircuits, thus further improving the performance, it also leads to a significantly high number of circuit executions, which may not be practical for most cases. Below, we show an example of the number of subcircuits corresponding to the number of cuts for a 50-qubit circuit.

![wc-5.png](../docs/images/tutorials/wire-cutting-to-improve-performance/5c6ea4da-bbd8-47f9-ac48-e438cc59a11d.avif)

Note that even for five cuts, the number of subexperiments is around 200k. Therefore, circuit cutting should be used only when the number of cuts is small.

### One example of cut-friendly and cut-unfriendly circuits each

#### Cut-friendly circuit

As noted earlier, a circuit is cut-friendly when the circuit can be partitioned into smaller disjoint subcircuits with a small number of cuts. Any hardware-efficient circuit, that is, a circuit that requires little to no SWAP gates when mapped to the hardware coupling map, is, in general, cut-friendly. Below, we show an example of an excitation preserving ansatz, which is used in Quantum Chemistry. Note that such a circuit can be partitioned into two subcircuits with a single cut irrespective of the number of qubits.

![wc-6.png](../docs/images/tutorials/wire-cutting-to-improve-performance/c122a418-b914-41e7-a1aa-00eb1eec5b87.avif)

#### Cut-unfriendly circuit

A circuit is cut-unfriendly if, in general, the number of cuts required to form disjoint partitions grow significantly with the depth of the number of qubits. Recall that with each cut an extra qubit is required. So with the number of cuts, the effective number of qubits also increase. Below we show an example of a 3-qubit Grover circuit with a possible cutting instance.

![wc-7.png](../docs/images/tutorials/wire-cutting-to-improve-performance/b31dc57e-e6d7-49fd-9f46-304a328b3764.avif)

We note that three cuts are required, and the cut is more vertical than horizontal. This means, that the number of cuts is expected to scale linearly with the number of qubits, which is not amenable for cutting.

## References


[1] Peng, T., Harrow, A. W., Ozols, M., & Wu, X. (2020). Simulating large quantum circuits on a small quantum computer. Physical review letters, 125(15), 150504.

[2] Tang, W., Tomesh, T., Suchara, M., Larson, J., & Martonosi, M. (2021, April). Cutqc: using small quantum computers for large quantum circuit evaluations. In Proceedings of the 26th ACM International conference on architectural support for programming languages and operating systems (pp. 473-486).

[3]  Perlin, M. A., Saleem, Z. H., Suchara, M., & Osborn, J. C. (2021). Quantum circuit cutting with maximum-likelihood tomography. npj Quantum Information, 7(1), 64.

[4]  Majumdar, R., & Wood, C. J. (2022). Error mitigated quantum circuit cutting. arXiv preprint arXiv:2211.13431.

[5]  Khare, T., Majumdar, R., Sangle, R., Ray, A., Seshadri, P. V., & Simmhan, Y. (2023). Parallelizing Quantum-Classical Workloads: Profiling the Impact of Splitting Techniques. In 2023 IEEE International Conference on Quantum Computing and Engineering (QCE) (Vol. 1, pp. 990-1000). IEEE.

[6]  Bhoumik, D., Majumdar, R., Saha, A., & Sur-Kolay, S. (2023). Distributed Scheduling of Quantum Circuits with Noise and Time Optimization. arXiv preprint arXiv:2309.06005.

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3BLFkNVEuh0QBWm)